<a href="https://colab.research.google.com/github/Arkhemis/Coursera_Capstone/blob/main/Capstone_ThirdAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neighborhoods battle in Toronto
We first import the relevant libraries

In [294]:
!pip install geocoder
import requests
import re
import pandas as pd
import numpy as np
import geocoder 
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

Then scraping the data from the Wikipedia page

In [295]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(wiki_url)
soup = BeautifulSoup(page.content, "html.parser")
toronto_table = soup.find('table')

We extract the data needed and transform it into a dataframe

In [296]:
dico = {'Postal_Code': [],
        'Borough': [],
        'Neighborhood': []
}
for data in toronto_table.find_all('td'):
  if data.span.text == "Not assigned":
    pass
  else:
    #Appending Postal_Code
    dico['Postal_Code'].append(data.find('p').text[:3])
    #Appending Neighborhood
    dico['Neighborhood'].append(re.findall(r"\(([^\)]+)\)", data.text))
    #Appending Borough
    br = (data.span.text).split('(')[0]
    dico['Borough'].append(br)
df = pd.DataFrame.from_dict(dico)

#Just converting the list into string
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: ' '.join(x))
print(df.shape)
df.head()

(103, 3)


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


Using the geocoder package (but alas, not working)

In [297]:

#coords = None
#df["latitude"] = 0
#df["longitude"] = 0

#for i in range(len(df)):
#  while(coords is None):
#    g = geocoder.google('{}, Toronto, Ontario'.format(df["Postal_Code"][i]))
#    print(g)
#    coords = g.latlng
 
# df["latitude"][i] = coords[0]
#  df["longitude"][i] = coords[1]

Since the above is not working, let us rely on the online csv and merge it with our dataset

In [298]:
location = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
location = location.rename(columns = {"Postal Code": "Postal_Code"})
df = df.merge(location.set_index('Postal_Code'), on='Postal_Code')
df.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [299]:
#Let's check the boroughs
print(df["Borough"].unique())

df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto',
                                             'EtobicokeNorthwest':'Etobicoke Northwest',
                                             'East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(df["Borough"].unique(), df["Borough"].value_counts())

['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East YorkEast Toronto'
 'Central Toronto' 'MississaugaCanada Post Gateway Processing Centre'
 'Downtown TorontoStn A PO Boxes25 The Esplanade' 'EtobicokeNorthwest'
 'East TorontoBusiness reply mail Processing Centre969 Eastern']
['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East York/East Toronto'
 'Central Toronto' 'Mississauga' 'Etobicoke Northwest'] North York                24
Downtown Toronto          18
Scarborough               17
Etobicoke                 11
Central Toronto            9
West Toronto               6
East Toronto               5
York                       5
East York                  4
Queen's Park               1
East York/East Toronto     1
Etobicoke Northwest        1
Mississauga                1
Name: Borough, dtype: int64


In [300]:
import folium
address = "Toronto"
#First, let's find the coordinates of Toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for latitude, longitude, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Pre-Processing

Here, we will be focusing on the Downtown Toronto borough.

In [301]:
CLIENT_ID = 'LIZWZBQQJFNW4XER0145UIZBUW5IBL4CRO3QOENC2025UR5G' 
CLIENT_SECRET = 'SPF1UKA5I1VIX5WBVEDGPHBGERY2NELCT1O1RA3TCKMI0R51' 
VERSION = '20180605' # Foursquare API Version
LIMIT = 100 

In [302]:
#Let's find the coordinates of the Toronto' Neighborhoods
df_downto = df[df['Borough'] == "Downtown Toronto"]
df_downto.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


Let's use this function to extract the informations regarding the venues in the different Neighborhoods

In [303]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's assign the venues to the neighborhoods

In [304]:
df_downto_venues = getNearbyVenues(df_downto.Neighborhood, df_downto.Latitude, df_downto.Longitude)
print("Shape:",df_downto_venues.shape)
df_downto_venues.head()

Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Rosedale
Enclave of M5E
St. James Town / Cabbagetown
First Canadian Place / Underground city
Church and Wellesley
Shape: (1211, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Now, let's regroup the venues by Neighborhoods

In [305]:
df_downto_venues.groupby('Neighborhood').count()
#We can count the unique categories as well
print("There is", len(df_downto_venues['Venue Category'].unique()), "unique categories in our dataset")

There is 210 unique categories in our dataset


Now, let's turn our categories into dummy (binary) variables

In [306]:
df_downto_dummy = pd.get_dummies(df_downto_venues[["Venue Category"]], prefix="")
df_downto_dummy["Neighborhood"] = df_downto_venues["Neighborhood"]
#Moving the column to the first position
df_downto_dummy.insert(0, 'Neighborhood', df_downto_dummy.pop('Neighborhood'))
df_downto_dummy.head()
df_downto_venues["Neighborhood"].unique()

array(['Regent Park / Harbourfront', 'Garden District, Ryerson',
       'St. James Town', 'Berczy Park', 'Central Bay Street', 'Christie',
       'Richmond / Adelaide / King',
       'Harbourfront East / Union Station / Toronto Islands',
       'Toronto Dominion Centre / Design Exchange',
       'Commerce Court / Victoria Hotel',
       'University of Toronto / Harbord',
       'Kensington Market / Chinatown / Grange Park',
       'CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport',
       'Rosedale', 'Enclave of M5E', 'St. James Town / Cabbagetown',
       'First Canadian Place / Underground city', 'Church and Wellesley'],
      dtype=object)

In [307]:
print("Before grouping by neighborhoods", df_downto_dummy.shape)
df_downto_grouped = df_downto_dummy.groupby('Neighborhood').mean().reset_index()
print("After grouping by neighborhoods", df_downto_grouped.shape)
df_downto_grouped.head()

Before grouping by neighborhoods (1211, 211)
After grouping by neighborhoods (18, 211)


,Neighborhood,_Adult Boutique,_Airport,_Airport Food Court,_Airport Gate,_Airport Lounge,_Airport Service,_Airport Terminal,_American Restaurant,_Antique Shop,_Aquarium,_Art Gallery,_Art Museum,_Arts & Crafts Store,_Asian Restaurant,_BBQ Joint,_Baby Store,_Bagel Shop,_Bakery,_Bank,_Bar,_Baseball Stadium,_Basketball Stadium,_Beach,_Bed & Breakfast,_Beer Bar,_Beer Store,_Belgian Restaurant,_Bistro,_Boat or Ferry,_Bookstore,_Boutique,_Brazilian Restaurant,_Breakfast Spot,_Brewery,_Bubble Tea Shop,_Building,_Burger Joint,_Burrito Place,_Business Service,...,_Rental Car Location,_Restaurant,_Roof Deck,_Sake Bar,_Salad Place,_Salon / Barbershop,_Sandwich Place,_Scenic Lookout,_Sculpture Garden,_Seafood Restaurant,_Shoe Store,_Shopping Mall,_Skating Rink,_Smoke Shop,_Smoothie Shop,_Snack Place,_Soup Place,_Spa,_Speakeasy,_Sporting Goods Shop,_Sports Bar,_Steakhouse,_Strip Club,_Supermarket,_Sushi Restaurant,_Taco Place,_Tailor Shop,_Taiwanese Restaurant,_Tanning Salon,_Tea Room,_Thai Restaurant,_Theater,_Theme Restaurant,_Trail,_Train Station,_Vegetarian / Vegan Restaurant,_Video Game Store,_Vietnamese Restaurant,_Wine Bar,_Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.017857,0.000000,0.0,0.0,0.0,0.000000,0.017857,0.053571,0.000000,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.000000,0.000000,0.000000,0.0,0.017857,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.035714,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.035714,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.017857,0.000000,0.0,0.000000,0.0,0.017857,0.0,0.0,0.000000,0.017857,0.000000,0.000000,0.0,0.0,0.017857,0.0,0.0,0.0,0.000000
1,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.055556,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.055556,0.000000,0.055556,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.055556,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.055556,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.014706,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.014706,0.014706,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.014706,0.000000,0.0,0.000000,0.0,0.029412,0.0,0.029412,0.000000,0.014706,...,0.000000,0.014706,0.0,0.000000,0.029412,0.000000,0.058824,0.0,0.014706,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.014706,0.0,0.000000,0.0,0.014706,0.000000,0.0,0.014706,0.0,0.000000,0.0,0.0,0.014706,0.014706,0.000000,0.000000,0.0,0.0,0.014706,0.0,0.0,0.0,0.014706
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.066667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Church and Wellesley,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.012658,0.0,0.000000,0.000000,0.0,0.012658,0.0,0.0,0.000000,0.000000,0.012658,0.000000,0.0,0.012658,0.0,0.025316,0.0,0.012658,0.012658,0.000000,...,0.000000,0.037975,0.0,0.012658,0.000000,0.012658,0.000000,0.0,0.012658,0.000000,0.0,0.0,0.0,0.012658,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.012658,0.012658,0.0,0.063291,0.0,0.000000,0.0,0.0,0.0000

# Top Venues by Neighborhoods
Let's first define a function that will return the top venues

In [308]:
def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [309]:
top_venues = 10

columns = ['Neighborhood']
#Let's create first the columns
for venue in range(top_venues):
  columns.append('{} Most Common Venue'.format(venue+1))
df_downto_sorted = pd.DataFrame(columns=columns)


df_downto_sorted['Neighborhood'] = df_downto_grouped['Neighborhood']

#Let's find the most common values now by Neighborhoods
for ind in np.arange(df_downto_grouped.shape[0]):
    df_downto_sorted.iloc[ind, 1:] = return_most_common_venues(df_downto_grouped.iloc[ind, :], top_venues)

df_downto_sorted.head()

,Neighborhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Berczy Park,_Bakery,_Coffee Shop,_Restaurant,_Pharmacy,_Farmers Market,_Seafood Restaurant,_Beer Bar,_Cheese Shop,_Cocktail Bar,_Italian Restaurant
1,CN Tower / King and Spadina / Railway Lands / ...,_Airport Service,_Airport Lounge,_Airport Terminal,_Sculpture Garden,_Airport,_Airport Food Court,_Airport Gate,_Bar,_Harbor / Marina,_Boutique
2,Central Bay Street,_Coffee Shop,_Sandwich Place,_Italian Restaurant,_Café,_Salad Place,_Bubble Tea Shop,_Burger Joint,_Japanese Restaurant,_Middle Eastern Restaurant,_Ice Cream Shop
3,Christie,_Grocery Store,_Café,_Park,_Baby Store,_Nightclub,_Candy Store,_Italian Restaurant,_Restaurant,_Coffee Shop,_Discount Store
4,Church and Wellesley,_Coffee Shop,_Japanese Restaurant,_Sushi Restaurant,_Restaurant,_Gay Bar,_Yoga Studio,_Pub,_Men's Store,_Mediterranean Restaurant,_Hotel


# Clustering
Now, we can cluster the neighborhoods, based on their features, i.e. their venues

In [310]:
from sklearn.cluster import KMeans
kclusters = 5

df_downto_cluster = df_downto_grouped.drop('Neighborhood', 1)

#Now, we train the model
kmeans = KMeans(n_clusters=kclusters, random_state=1962).fit(df_downto_cluster)

#Let's check the first labels
print(kmeans.labels_[0:10])
df_downto_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

[2 4 2 3 2 2 2 2 2 2]


In [311]:
df_downto_merged = df_downto

df_downto_merged = df_downto.join(df_downto_sorted.set_index('Neighborhood'), on='Neighborhood')
df_downto_merged["Cluster_Labels"] = pd.to_numeric(df_downto_merged["Cluster_Labels"], downcast='integer')
print(df_downto_merged.shape)
#Drop NAs in the case they are neighborhoods that do not have any venues
df_downto_merged = df_downto_merged[df_downto_merged["1 Most Common Venue"].notna()]

df_downto_merged.head() 

(18, 16)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,2,_Coffee Shop,_Pub,_Bakery,_Park,_Breakfast Spot,_Theater,_Café,_Yoga Studio,_Event Space,_Performing Arts Venue
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,_Coffee Shop,_Clothing Store,_Bubble Tea Shop,_Japanese Restaurant,_Hotel,_Middle Eastern Restaurant,_Cosmetics Shop,_Café,_Theater,_Diner
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,_Coffee Shop,_Café,_Restaurant,_Cosmetics Shop,_Cocktail Bar,_Italian Restaurant,_Farmers Market,_Seafood Restaurant,_Beer Bar,_Park
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,_Bakery,_Coffee Shop,_Restaurant,_Pharmacy,_Farmers Market,_Seafood Restaurant,_Beer Bar,_Cheese Shop,_Cocktail Bar,_Italian Restaurant
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,_Coffee Shop,_Sandwich Place,_Italian Restaurant,_Café,_Salad Place,_Bubble Tea Shop,_Burger Joint,_Japanese Restaurant,_Middle Eastern Restaurant,_Ice Cream Shop


# Vizualisation 

Now, we can visualise the newly created clusters on the map

In [313]:
import matplotlib.cm as cm
import matplotlib.colors as colors
address = "Downtown Toronto, Toronto"
#First, let's find the coordinates of Toronto
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

#Let's set the clusters' colors
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_downto_merged['Latitude'], df_downto_merged['Longitude'], df_downto_merged['Neighborhood'], df_downto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters